# Importing libraries

Because the original cancer datasets do not contain annotations, the effect of joint dimensionality reduction approaches on factors with clinical annotations or biological annotations does not need to be considered when processing such datasets. Just need to consider the effect of factors on survival.

In general, `MCIA`, `RGCCA`, and `JIVE` achieved the best performances, finding factors significantly associated with survival in seven out of ten cancer types.

In [1]:
# MCIA is implemented in the R package omicade4
# (https://bioconductor.org/packages/release/bioc/html/omicade4.html).
library("omicade4")

# RGCCA is implemented into the CRAN package RGCCA
# (https://cran.r-project.org/web/packages/RGCCA/index.html).
library("RGCCA")

# JIVE is implemented into the R package r.jive
# (https://cran.r-project.org/web/packages/r.jive/index.html).
library("r.jive")

Loading required package: ade4



# Importing functions

In [2]:
# The function runfactoization runs all the considered multi-omics factorization.
runfactorization <- function(folder,file.names,num.factors,sep=" ",filtering="none"){
    factorizations<-list()
    t<-1
    method<-numeric(0)
    
    num.factors<-as.numeric(num.factors)

    
    ##creating list of omics
    omics <- list()
    for(i in 1:length(file.names)){
      omics[[i]]<-as.matrix(read.table(paste(folder,file.names[i],sep="/"),sep=sep,row.names=1,header=T))
    }
    
    ####
    #omics<-lapply(omics, function(x) t(x))
    ######
    
    ##restricting to common samples and filtering
    samples<-colnames(omics[[1]])
    for(j in 1:length(omics)){
      samples<-intersect(samples,colnames(omics[[j]]))
    }
    for(j in 1:length(omics)){
      omics[[j]]<-omics[[j]][,samples]
      if(filtering!="none"){
        x<-apply( omics[[j]],1,sd)
        x<-as.matrix(sort(x, decreasing = T))
        w<-which(x>0)
        if(filtering=="stringent"){
          selected<-rownames(x)[1:min(w[length(w)],5000)]
        }else{
          selected<-rownames(x)[1:min(w[length(w)],6000)]
        }
        m<-match(rownames(omics[[j]]),selected)
        w<-which(!is.na(m))
        omics[[j]]<-omics[[j]][w,]
      }else{
        omics[[j]]<-omics[[j]][,which(apply(omics[[j]],2,sd)>0)]
      }
    }  

    ##RGCCA 
    factorizations_RGCCA<-rgcca(lapply(omics, function(x) t(x)), ncomp = rep(num.factors, length(omics)), scheme = "centroid", scale = TRUE, init = "svd",bias = TRUE, tol = 1e-08, verbose = F)
    factors_rgcca<-as.matrix(factorizations_RGCCA$Y[[1]])
    metagenes_rgcca <- list()
    for(j in 1:length(omics)){
      metagenes_rgcca[[j]]<-as.matrix(factorizations_RGCCA$a[[j]])
      rownames(metagenes_rgcca[[j]])<-rownames(omics[[j]])
      colnames(metagenes_rgcca[[j]])<-1:num.factors
    }
    factorizations[[t]]<-list(factors_rgcca,metagenes_rgcca)
    t<-t+1
    method<-c(method,"RGCCA")
    
    ###MCIA
    omics_pos<-list()
    for(j in 1:length(omics)){
      if(min(omics[[j]])<0){
        omics_pos[[j]]<-omics[[j]]+abs(min(omics[[j]]))
      }else{
        omics_pos[[j]]<-omics[[j]]
      }
      omics_pos[[j]]<-omics_pos[[j]]/max(omics_pos[[j]])
    }
    factorizations_mcia<-mcia(omics_pos, cia.nf = num.factors)
    factors_mcia<-as.matrix(factorizations_mcia$mcoa$SynVar)
    metagenes_mcia<-list()
    for(j in 1:length(omics)){
      metagenes_mcia[[j]]<-as.matrix(factorizations_mcia$mcoa$axis[1:dim(omics[[j]])[1],])
      rownames(metagenes_mcia[[j]])<-rownames(omics[[j]])
      colnames(metagenes_mcia[[j]])<-1:num.factors
    }
    factorizations[[t]]<-list(factors_mcia,metagenes_mcia)
    t<-t+1
    method<-c(method,"MCIA")

    ### JIVE
    factorizations_jive<-jive(omics, rankJ=num.factors, rankA = rep(num.factors, length(omics)), method = "given", conv = "default", maxiter = 100, showProgress=FALSE)
    rankJV <- factorizations_jive$rankJ;
    rankIV.v <- factorizations_jive$rankA;
    J<-numeric(0)
    ng<-0
    metagenes_jive <- list();
    for(j in 1:length(omics)){
      J <- rbind(J,factorizations_jive$joint[[j]]);
      ng<-c(ng,dim(factorizations_jive$joint[[j]])[1])
    }
    svd.o <- svd(J);
    jV <- svd.o$v %*% diag(svd.o$d);
    for(j in 1:length(omics)){
      metagenes_jive[[j]] <- svd.o$u[(1+sum(ng[1:j])):sum(ng[1:j+1]),1:rankJV]; ###error in dimension
      rownames(metagenes_jive[[j]])<-rownames(omics[[j]])
      colnames(metagenes_jive[[j]])<-1:num.factors
    }
    factors_jive=jV[,1:rankJV]
    rownames(factors_jive)<-colnames(omics[[1]])
    colnames(factors_jive)<-1:num.factors
    factorizations[[t]]<-list(factors_jive,metagenes_jive)
    t<-t+1
    method<-c(method,"JIVE")
                                       
    out<-list(factorizations=factorizations,method=method)
    
    return(out)
}

In [3]:
# The input files need to be log2 transformed before running the analysis.
# Support function to apply log2(+1) to a matrix.
log2matrix <- function(folder, file.name){
    # Read table
    data <- as.matrix(read.table(paste(folder,file.name,sep="/"),sep=" ",row.names=1,header=TRUE))
    # Apply transformation
    data <- log2(data+1)
    # Output file name
    output <- paste(folder,paste0("log_",file.name), sep="/")
    # Export transformed data
    write.table(data,output,sep=" ", col.names=TRUE, row.names=TRUE)
    # ?
    system(paste("sed -i '1s/^/probe\t/'", output, sep=" "))
}

# Loading data

Between the five multi-omics cancer datasets, I'd live to choose `BIC` (dataset of breast), `LIHC` (dataset of the liver) and `SKCM` (dataset of melanoma). Because `MCIA` performed the best for melanoma; `JIVE` performed the best in liver cancer ; `RGCCA` performed the best in breast cancer.

In [4]:
cancers <- c('./data/cancer/breast',
             './data/cancer/liver',
             './data/cancer/melanoma')

# Saving results

In [5]:
# Label to identify current run
tag <- format(Sys.time(), "%Y_%m_%d_%H_%M")

# Folder for comparison results
results_folder <- paste0("./results_", tag, "/")

# Create output folder
dir.create(results_folder, showWarnings = FALSE)

In [6]:
# Number of factors used in the paper
num.factors <- 2

In [7]:
for(i in cancers){

    print(paste0("Now analysing ", i))
    
    # Name of current cancer
    current_cancer <- basename(i)

    # If the expression and miRNA data are not log2-transformed as for those provided by XX et al.
    log2matrix(i,"exp")
    log2matrix(i,"mirna")

    # Perform factorisation
    print("Running factorisation...")
    out <- runfactorization(i, c("log_exp","methy","log_mirna"), num.factors, sep=" ", filtering="sd")
    save(out, file=paste0(results_folder, current_cancer, "_results_out.rds"))
    
}

[1] "Now analysing ./data/cancer/breast"
[1] "Running factorisation..."
[1] "Now analysing ./data/cancer/liver"
[1] "Running factorisation..."
[1] "Now analysing ./data/cancer/melanoma"
[1] "Running factorisation..."
